Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [477]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os # use this to access your environment variables
import requests # this will be used to call the APIs
from unidecode import unidecode # remove accents from venues e.g. café --> cafe
import re # regex search for strings


FOURSQUARE_KEY = os.getenv('FOURSQUARE_API_KEY')
YELP_KEY = os.getenv('YELP_API_KEY')

In [479]:
bike_stations = pd.read_csv("../data/bike_stations_sat_3pm_clean.csv")

**This bike_stations data is based on the Saturday, 3pm citybikes API call**

In [522]:
bike_stations

,id,name,latitude,longitude,free_bikes,empty_slots,bike_availability,ll
0,fb337bbed72e2be090071e199899b2be,Queen St E / Woodward Ave,43.665269,-79.319796,18,1,94.74,"43.665269,-79.319796"
1,4ff88d5880e71aa40d34cfe5d09b0ca7,Primrose Ave / Davenport Rd,43.671420,-79.445947,1,14,6.67,"43.67142,-79.445947"
2,a09c67c0b419654d907c9134b108e328,Queen St E / Rhodes Ave,43.666224,-79.317693,12,11,52.17,"43.666224,-79.317693"
3,d6a9daee68070a8b106cfb598d81308c,Bond St / Queen St E,43.653236,-79.376716,5,32,13.51,"43.653236,-79.376716"
4,8f8af40d9388c8a3962559e8681d3db7,Church St / Alexander St,43.663722,-79.380288,3,32,8.57,"43.663722,-79.380288"
...,...,...,...,...,...,...,...,...
821,9be5f078a1ed47fc11cd3cee45260f63,Kennedy Rd/Ranstone Gdns (Jack Goodlad Park),43.741906,-79.271819,1,10,9.09,"43.741906,-79.271819"
822,4ae37f3bddfb819954a15143d277dbd9,Eglinton Ave E / Brimley Rd,43.736953,-79.247984,8,11,42.11,"43.736953,-79.247984"
823,e7968ab22d9a15db0673f463144428eb,College Park South,43.659457,-79.382365,7,11,38.89,"43.659457,-79.382365"
824,62acc308c0f93ff09d28e06c73afc3ec,165 McRae Dr,43.705875,-79.368006,4,11,26.67,"43.705875,-79.368006"


In [104]:
# testing the ability to parse for POI category, number of POIs for a sample API response, need the non-cleaned version
venue_df = pd.read_csv("../data/test_venue_df.csv")

In [518]:
#sample_bike_df = bike_stations.sample(n=100)

In [520]:
#sample_bike_df

**Strategy is as follows:
Steps for the 10 stations in sample_bike_df, ensure it works for the 820 rows in the whole city bike dataset**:

1. Define my get_venues_fs function to call the Foursquare API. Set it up so that it can take a concatenated ll string, like the ll column of bike_stations and sample_bike_df.
2. Initialize a new bike_stations df with the columns to fill in for # of POIs, and # of POIs in certain categories.
3. Loop through each bike station, make the get_venues_fs call with its ll column to find venue results with the data in step 2. 
4. With the returned results, we'll parse for the number of POI categories, and the number of POIs generally (know that the upper limit of returns is 50, radius of the call is 800m to make <=50 results meaningful)
5. We should have a dataframe returned containing 1) the original ll call which we can use to join the bike_stations table later if we wish, 2) the number of POIs and number of establishments within POI categories, to train the model

Notes:

**I am doing step 2 here and not in yelp_foursquare_EDA to avoid bringing city_bikes data into the yelp_foursquare notebook, so any cross-referencing of city bikes with venue data will happen here**.

**In the likely event that some bike station ll's will retrieve establishments that were retrieved by another bike station (e.g. in Downtown Toronto where several stations will be within 800m of each other), I am retrieving the fsq_id for the establishments which will be checked for dupes, with dupes removed, before removing the column in cleaning to join it with a cleaned bike_stations df**

In [461]:
# Step 1 from Strategy: Defining the foursquare API Call here

def get_venues_fs(ll, radius, api_key, categories, limit):
    """
    Get amenities and POIs from Foursquare API call
    Args:
        ll (str): concatenated latitude,longitude that is used to supply the API and return POIs
        radius: number marking the radius in metres for the POIs to be collected from
        api_key (str): foursquare API to use for query (imported above)
        categories (str) : Foursquare-recognized place type. four categories provided at bottom of this cell
        limit (int): will be given as 50, the maximum for this API
    
    Returns:
        response: response object from the requests library.
    """
    url = "https://api.foursquare.com/v3/places/search"
    
    headers = {
        "Accept": "application/json",
        "Authorization": api_key
    }
    
    params = {
        "ll": ll,
        "radius": radius,
        "categories": categories,
        "limit": limit
    }
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200: # 200 is success
        return response.json()
    else:
        response.raise_for_status()

categories = '10035,13003,13065,16000' # Category codes - bars, restaurants, live shows, outdoors


In [524]:
# Step 2: get the bike station dataframe ready for the Foursquare API data to fill in in step 3

bike_stations['n_pois'] = 0
bike_stations['n_bar_restaurant'] = 0
bike_stations['n_cafe'] = 0
bike_stations['n_live'] = 0
bike_stations['n_park'] = 0
bike_stations

,id,name,latitude,longitude,free_bikes,empty_slots,bike_availability,ll,n_pois,n_bar_restaurant,n_cafe,n_live,n_park
0,fb337bbed72e2be090071e199899b2be,Queen St E / Woodward Ave,43.665269,-79.319796,18,1,94.74,"43.665269,-79.319796",0,0,0,0,0
1,4ff88d5880e71aa40d34cfe5d09b0ca7,Primrose Ave / Davenport Rd,43.671420,-79.445947,1,14,6.67,"43.67142,-79.445947",0,0,0,0,0
2,a09c67c0b419654d907c9134b108e328,Queen St E / Rhodes Ave,43.666224,-79.317693,12,11,52.17,"43.666224,-79.317693",0,0,0,0,0
3,d6a9daee68070a8b106cfb598d81308c,Bond St / Queen St E,43.653236,-79.376716,5,32,13.51,"43.653236,-79.376716",0,0,0,0,0
4,8f8af40d9388c8a3962559e8681d3db7,Church St / Alexander St,43.663722,-79.380288,3,32,8.57,"43.663722,-79.380288",0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
821,9be5f078a1ed47fc11cd3cee45260f63,Kennedy Rd/Ranstone Gdns (Jack Goodlad Park),43.741906,-79.271819,1,10,9.09,"43.741906,-79.271819",0,0,0,0,0
822,4ae37f3bddfb819954a15143d277dbd9,Eglinton Ave E / Brimley Rd,43.736953,-79.247984,8,11,42.11,"43.736953,-79.247984",0,0,0,0,0
823,e7968ab22d9a15db0673f463144428eb,College Park South,43.659457,-79.382365,7,11,38.89,"43.659457,-79.382365",0,0,0,0,0
824,62acc308c0f93ff09d28e06c73afc3ec,165 McRae Dr,43.705875,-79.368006,4,11,26.67,"43.705875,-79.368006",0,0,0,0,0


In [526]:
# Step 3:

# lists for the augmented bike_stations for the model - number of POIs, bars/restaurants/live venues, parks to populate columns for the model training
n_poi_list = [] # this is solved by n_poi_list.append(len(res['results']))
n_restobar_list = []
n_live_list = []
n_park_list = []
n_cafe_list = []

# lists for the venues returned via the API call

fsq_id_list = []
ll_list = [] # this is here for the ll from the bike station that 'summoned' this venue. For joining the SQL tables.
venue_ll_list = [] # this is the ll for the venue itself.
name_list = []
address_list = []
category_list = []

# set up regex patterns for finding # of bar/restaurant, cafes, parks, live venues to search within the category column. These are loose 'contain' matches
bar_restaurant_pattern = re.compile(r'\b(bar|restaurant|gastropub|brewery|lounge|bbq|pub|grill|burger|chicken|ramen|diner|Fish and Chips|pizzeria|tavern|night club|bistro|steakhouse|nightclub)\b', re.IGNORECASE)
live_venue_pattern = re.compile(r'\b(concert|music venue|music|comedy|live|theater|History Museum)\b', re.IGNORECASE)
park_pattern = re.compile(r'\b(park|playground|monument|plaza|structure|Historic|Protected Site|Outdoors|Lake|campground|Landmarks|Scenic Lookout|Outdoors|monument|garden|beach|Hiking|Trail)\b', re.IGNORECASE)
cafe_pattern = re.compile(r'\b(cafe|coffee|coffee shop|tea|bakery|ice cream|donut|deli|dessert|bagel|sandwich)\b', re.IGNORECASE)


In [528]:
# Step 3 and Step 4:

for index, row in bike_stations.iterrows():
    n_restobar = 0 # I will iterate over each venue returned for the bike station's coordinates and ticker up for a positive match to the regex
    n_live = 0
    n_park = 0
    n_cafe = 0
    lat_long = row['ll']
    
    res = get_venues_fs(ll=lat_long, radius=800, api_key=FOURSQUARE_KEY, categories=categories, limit=50) # here's the API call to get the venues
    
    n_poi_list.append(len(res['results'])) # the number of POIs is simply length of the results, no need to ticker.

    # step 4 - populating the df with POIs from the api call
    
    for venue in res['results']:
        fsq_id_list.append(venue['fsq_id'])
        ll_list.append(lat_long)
        venue_ll_list.append(f"{venue['geocodes']['main']['latitude']},{venue['geocodes']['main']['longitude']}")
        name_list.append(venue['name'])
        address_list.append(venue['location'].get('address', 'Address not available'))
        category_list.append(venue['categories'][0]['name'])
        
        category_name = venue['categories'][0]['name'] # capturing the first category name in order to regex check it for one of our major categories of interest
        category_name = unidecode(category_name)
        
        if bar_restaurant_pattern.search(category_name): # Regex statements defined in cell above
            n_restobar += 1
        elif live_venue_pattern.search(category_name):
            n_live += 1
        elif park_pattern.search(category_name):
            n_park += 1
        elif cafe_pattern.search(category_name):
            n_cafe += 1

    n_restobar_list.append(n_restobar)
    n_live_list.append(n_live)
    n_park_list.append(n_park)
    n_cafe_list.append(n_cafe)
    

In [511]:
print(n_poi_list)
print(n_restobar_list)
print(n_live_list)
print(n_park_list)
print(n_cafe_list)

#print(f"{row['name']}, {row['ll']}, {row['n_pois']}") - just a spare print for the cell above

[]
[]
[]
[]
[]


In [530]:
bike_stations['n_pois'] = n_poi_list
bike_stations['n_bar_restaurant'] = n_restobar_list
bike_stations['n_live'] = n_live_list
bike_stations['n_park'] = n_park_list
bike_stations['n_cafe'] = n_cafe_list

In [539]:
bike_stations

,id,name,latitude,longitude,free_bikes,empty_slots,bike_availability,ll,n_pois,n_bar_restaurant,n_cafe,n_live,n_park
0,fb337bbed72e2be090071e199899b2be,Queen St E / Woodward Ave,43.665269,-79.319796,18,1,94.74,"43.665269,-79.319796",42,24,4,3,9
1,4ff88d5880e71aa40d34cfe5d09b0ca7,Primrose Ave / Davenport Rd,43.671420,-79.445947,1,14,6.67,"43.67142,-79.445947",49,36,3,1,8
2,a09c67c0b419654d907c9134b108e328,Queen St E / Rhodes Ave,43.666224,-79.317693,12,11,52.17,"43.666224,-79.317693",50,29,6,3,10
3,d6a9daee68070a8b106cfb598d81308c,Bond St / Queen St E,43.653236,-79.376716,5,32,13.51,"43.653236,-79.376716",50,32,6,5,5
4,8f8af40d9388c8a3962559e8681d3db7,Church St / Alexander St,43.663722,-79.380288,3,32,8.57,"43.663722,-79.380288",50,34,9,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
821,9be5f078a1ed47fc11cd3cee45260f63,Kennedy Rd/Ranstone Gdns (Jack Goodlad Park),43.741906,-79.271819,1,10,9.09,"43.741906,-79.271819",3,0,0,0,3
822,4ae37f3bddfb819954a15143d277dbd9,Eglinton Ave E / Brimley Rd,43.736953,-79.247984,8,11,42.11,"43.736953,-79.247984",32,23,4,0,4
823,e7968ab22d9a15db0673f463144428eb,College Park South,43.659457,-79.382365,7,11,38.89,"43.659457,-79.382365",50,33,6,4,6
824,62acc308c0f93ff09d28e06c73afc3ec,165 McRae Dr,43.705875,-79.368006,4,11,26.67,"43.705875,-79.368006",50,37,8,0,4


I noticed that for 50 POIs the sum of restobar, cafe, live, park was sometimes only coming to about 45 or 40, I went back to check which other tags were not being picked up by my Regex statements:
- Structure, e.g. the famous Casa Loma building in Toronto
- Lake
- Historic and Protected Site
- Landmarks and Outdoors

I noticed others such as Tree Service and Bridge but I judged via the names and checking on Google maps that these are not necessarily points of interest and may be more related to key infrastructure and maintenance, which probably does not influence the bike station availability. So, whilst I captured some extra key categories missed in my initial regex by taking random bike station samples, I am happy with not every single POI under the Foursquare categories being captured by my regex statements.

In [534]:
# set up a dataframe for all the venues that we collected from the Foursquare API Call

venues_dict = {
    'foursquare_id': fsq_id_list,
    'll': ll_list,
    'venue_ll': venue_ll_list,
    'name': name_list,
    'address': address_list,
    'category': category_list
}

#print(fsq_id_list, ll_list, name_list, address_list, category_list)

venues_df = pd.DataFrame(venues_dict)
venues_df


,foursquare_id,ll,venue_ll,name,address,category
0,58d4650e2f91cb5ccec03447,"43.665269,-79.319796","43.663372,-79.319902",Rorschach Brewing,1001 Eastern Ave,Brewery
1,55667a68498e74fae3631f5e,"43.665269,-79.319796","43.664574,-79.325048",The Sidekick,1374 Queen St E,Coffee Shop
2,4baa029cf964a52085433ae3,"43.665269,-79.319796","43.662892,-79.312532",Woodbine Park,Queen St,Park
3,5859d15c1d21ba53f2f9b4ef,"43.665269,-79.319796","43.672346,-79.320804",Lake Inez,1471 Gerrard St E,Bar
4,61887b2d4467866f56d0f1e0,"43.665269,-79.319796","43.666823,-79.313292",History,1663 Queen St E,Concert Hall
...,...,...,...,...,...,...
33626,4bcce06fcc8cd13a41c4c1cf,"43.673489,-79.413395","43.668966,-79.40632",Gwendolyn MacEwen Park,Lowther Ave,Landmarks and Outdoors
33627,4ccc93662dc43704cb8dc808,"43.673489,-79.413395","43.666578,-79.411364",Ed and Anne Mirvish Parkette,843 Bathurst,Park
33628,0855e353488048218636b0fd,"43.673489,-79.413395","43.671328,-79.413994",National Youth Orchestra of Canada,1032 Bathurst St,Arts and Entertainment
33629,a5c4c27dd1a74d408546e098,"43.673489,-79.413395","43.676124,-79.415977",Country Style Donuts,1138 Bathurst St,Bagel Shop


**Success!** 

We have all 820 bike stations with the POI data and 33,600 rows of venue data in a data frame. We'll save both as CSVs in the data directory and save them to a SQL database and build the model with the bike_stations dataframe

In [543]:
# saving a file path for the model dataset - all done on the Saturday, 3pm bike stations dataset

csv_file_path = '../data/bike_stations_w_venues.csv' # this is the full bike stations with venue / POI data added now saved to my repository locally
bike_stations.to_csv(csv_file_path, index=False)

In [545]:
# saving a file path for the venue data

csv_file_path = '../data/full_venue_df.csv' # this is the full venue dataframe, 33600 records to be put in a SQL table later
venues_df.to_csv(csv_file_path, index=False)

In [ ]:
# Json normalize!!!

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

Look at the data before and after the join to validate your data.